Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [24]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [0]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [0]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0 : 3.51275
Minibatch accuracy: 6.2%
Validation accuracy: 12.8%
Minibatch loss at step 50 : 1.48703
Minibatch accuracy: 43.8%
Validation accuracy: 50.4%
Minibatch loss at step 100 : 1.04377
Minibatch accuracy: 68.8%
Validation accuracy: 67.4%
Minibatch loss at step 150 : 0.601682
Minibatch accuracy: 68.8%
Validation accuracy: 73.0%
Minibatch loss at step 200 : 0.898649
Minibatch accuracy: 75.0%
Validation accuracy: 77.8%
Minibatch loss at step 250 : 1.3637
Minibatch accuracy: 56.2%
Validation accuracy: 75.4%
Minibatch loss at step 300 : 1.41968
Minibatch accuracy: 62.5%
Validation accuracy: 76.0%
Minibatch loss at step 350 : 0.300648
Minibatch accuracy: 81.2%
Validation accuracy: 80.2%
Minibatch loss at step 400 : 1.32092
Minibatch accuracy: 56.2%
Validation accuracy: 80.4%
Minibatch loss at step 450 : 0.556701
Minibatch accuracy: 81.2%
Validation accuracy: 79.4%
Minibatch loss at step 500 : 1.65595
Minibatch accuracy: 43.8%
Validation accuracy: 79.6%

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [43]:
# Net work structure:
# 2 convolutional layers = Conv. layer + ReLU + Pooling
# 1 fully connected layer

# Parameter
learning_rate = 0.0001
epochs = 10
batch_size = 128  # decrease if not enough memory

test_valid_size = 5000

# Network param
filter_1_size = 5
filter_2_size = 5

# choose SAME padding & value of stride so that after each Conv layer, output is half of input size
graph = tf.Graph()
with graph.as_default():
    # define input to model
    tf_train_dataset = tf.placeholder(tf.float32, [None, image_size, image_size, num_channels])
    tf_train_labels = tf.placeholder(tf.float32, [None, num_labels])
    tf_valid_dataset = tf.constant(valid_dataset[:test_valid_size, :, :, :])
    mini_valid_labels = valid_labels[:test_valid_size, :]
    tf_test_dataset = tf.constant(test_dataset[:test_valid_size, :, :, :])
    mini_test_labels = test_labels[:test_valid_size, :]
    
    # weight & biases
    #[filter_height, filter_width, filter_depth, num_filters]
    weights = {
        'wc1' : tf.Variable(tf.random_normal([filter_1_size, filter_1_size, 1, 32])),  
        'wc2' : tf.Variable(tf.random_normal([filter_2_size, filter_2_size, 32, 64])),
        'wfc' : tf.Variable(tf.random_normal([7 * 7 * 64, num_labels]))}

    biases = {
        'bc1' : tf.Variable(tf.zeros([32])),
        'bc2' : tf.Variable(tf.zeros([64])),
        'bfc' : tf.Variable(tf.zeros([num_labels]))}


    def conv2d(x, W, b, stride_size=1):
        x = tf.nn.conv2d(x, W, strides=[1, stride_size, stride_size, 1], padding='SAME')
        x = tf.nn.bias_add(x, b)
    
        return tf.nn.relu(x)


    def maxPool2d(x, neighbor_size=2, stride_size=2):
        return tf.nn.max_pool(x,
                             ksize=[1, neighbor_size, neighbor_size, 1],
                             strides=[1, stride_size, stride_size, 1],
                             padding='SAME')


    def model(x, weights, biases, dropout):
        # Layer 1: 28x28x1 -> 14x14x32
        conv1 = conv2d(x, weights['wc1'], biases['bc1'])
        conv1 = maxPool2d(conv1)
    
        # introduce dropout
        conv1 = tf.nn.dropout(conv1, dropout)
    
        # Layer 2: 14x14x32 -> 7x7x64
        conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
        conv2 = maxPool2d(conv2)
    
        # Layer 3: 7x7x64 -> num_lablesx1
        fc = tf.reshape(conv2, [-1, weights['wfc'].get_shape().as_list()[0]])  
        fc = tf.matmul(fc, weights['wfc']) + biases['bfc']  # this is the logit
    
        return fc
    
    # Training computation
    logits = model(tf_train_dataset, weights, biases, 0.5)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset, weights, biases, 1))
    test_prediction = tf.nn.softmax(model(tf_test_dataset, weights, biases, 1))
    

In [44]:
# def accuracy(prediction, labels):
#     correct_pred = tf.equal(tf.argmax(prediction), tf.argmax(labels))
#     acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
#     return tf.to_float(acc)

num_steps = 300
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
        if (step % 50 == 0):
            print('Step %d:\tLoss:%.2f\t\tTrain Acc:%.1f\t\tValid Acc:%.1f' % 
                  (step, 
                   l, 
                   accuracy(predictions, batch_labels), 
                   accuracy(valid_prediction.eval(), mini_valid_labels)))
            
    print('Test accuracy: %.1f%%' % (accuracy(test_prediction.eval(), mini_test_labels)))



Initialized
Step 0:	Loss:4792.46		Train Acc:14.1		Valid Acc:13.0
Step 50:	Loss:2260.38		Train Acc:12.5		Valid Acc:15.9
Step 100:	Loss:1438.74		Train Acc:27.3		Valid Acc:23.7
Step 150:	Loss:1287.93		Train Acc:28.1		Valid Acc:29.0
Step 200:	Loss:1046.61		Train Acc:35.2		Valid Acc:32.4
Step 250:	Loss:881.08		Train Acc:28.9		Valid Acc:34.5
Test accuracy: 40.3%


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---